### Lemmatizacion 

#### Instalación y Obtención de datos

- Instalaciones 

In [65]:
!pip install spacy wordcloud matplotlib scikit-learn
!pip install nltk
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


- Importaciónes 

In [66]:
import pandas as pd
import spacy
import nltk
import re
from nltk.corpus import stopwords, wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt

- Descargar recursos NLTK

In [67]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Cargar los recursos de spaCy

In [68]:
# Cargar modelos de spaCy
nlp_es = spacy.load('es_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')
stopwords_es = set(stopwords.words('spanish'))
stopwords_en = set(stopwords.words('english'))

lemmatizer_en = WordNetLemmatizer()

- Obtención de datos

In [69]:
# Ruta al archivo CSV
ruta_csv = './reviews_multilang.csv'

# Cargar los datos correctamente especificando el separador
df = pd.read_csv(
    ruta_csv,
    sep=',',
    engine='python',      # Usamos el motor más flexible
    on_bad_lines='skip'   # Esta es la opción válida actualmente
)

# Mostrar las primeras filas
df.head()

,texto,idioma,sentimiento
0,La interfaz intuitiva facilita la navegación i...,es,positivo
1,El rendimiento gráfico dejó mucho que desear e...,es,negativo
2,Constantes fallos de conectividad WiFi tras la...,es,negativo
3,Sorprendido por la autonomía de la batería en ...,es,positivo
4,El sistema operativo se bloquea recurrentement...,es,negativo


- Crear la función lematizar(texto, idioma)

In [70]:
def nltk_pos_tag_to_wordnet(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lematizar(texto, idioma):
    if idioma == 'es':
        doc = nlp_es(texto.lower())
        return ' '.join([token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stopwords_es])
    elif idioma == 'en':
        tokens = word_tokenize(texto.lower())
        tagged = pos_tag(tokens)
        lemas = [lemmatizer_en.lemmatize(token, nltk_pos_tag_to_wordnet(tag)) for token, tag in tagged]
        return ' '.join([lema for lema in lemas if lema.isalpha() and lema not in stopwords_en])
    else:
        return texto


- Implemetancion del Lematizador

In [71]:
df['lemas'] = df.apply(lambda fila: lematizar(fila['texto'], fila['idioma']), axis=1)

#### Fase 2: Detección de Términos Técnicos y Jerga

In [72]:
# Lista personalizada de jergas técnicas
jerga_tecnica = {
    r'\bcrashe[oó]\b': 'crashear',
    r'\bbuguead[oa]\b': 'bugueado',
    r'\bfreez(e|ed|ing)\b': 'freeze',
    r'\bglitche[ds]?\b': 'glitch',
    r'\blague[ao]\b': 'lag'
}

def normalizar_jerga(texto):
    for patron, reemplazo in jerga_tecnica.items():
        texto = re.sub(patron, reemplazo, texto, flags=re.IGNORECASE)
    return texto

# Aplicar normalización a columna de lemas
df['lemas'] = df['lemas'].apply(normalizar_jerga)

#### Fase 3: Modelo de Clasificación y Comparación

In [ ]:
# Codificar el target
df['sentimiento_binario'] = df['sentimiento'].map({'positivo': 1, 'negativo': 0})

# Comparar usando texto original vs texto lematizado
X_raw = df['texto']
X_lem = df['lemas']
y = df['sentimiento_binario']

# Vectorizar y dividir
vectorizer_raw = TfidfVectorizer()
vectorizer_lem = TfidfVectorizer()

X_raw_vec = vectorizer_raw.fit_transform(X_raw)
X_lem_vec = vectorizer_lem.fit_transform(X_lem)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw_vec, y, test_size=0.2, random_state=42)
X_train_lem, X_test_lem, _, _ = train_test_split(X_lem_vec, y, test_size=0.2, random_state=42)

# Entrenar y evaluar
clf_raw = RandomForestClassifier(random_state=42)
clf_lem = RandomForestClassifier(random_state=42)

clf_raw.fit(X_train_raw, y_train)
clf_lem.fit(X_train_lem, y_train)

pred_raw = clf_raw.predict(X_test_raw)
pred_lem = clf_lem.predict(X_test_lem)

f1_raw = f1_score(y_test, pred_raw)
f1_lem = f1_score(y_test, pred_lem)

print("F1-score SIN lematización:", f1_raw)
print("F1-score CON lematización:", f1_lem)

# Tamaño de vocabulario
print("Tamaño vocabulario original:", len(vectorizer_raw.vocabulary_))
print("Tamaño vocabulario lematizado:", len(vectorizer_lem.vocabulary_))

F1-score SIN lematización: 0.6
F1-score CON lematización: 0.5714285714285714
Tamaño vocabulario original: 276
Tamaño vocabulario lematizado: 243


#### Fase 4: Evaluación Comparativa

In [74]:
# Reducción de vocabulario
vocab_reduccion = 100 * (1 - len(vectorizer_lem.vocabulary_) / len(vectorizer_raw.vocabulary_))
print(f"Reducción del vocabulario: {vocab_reduccion:.2f}%")

# Mejora de F1
f1_mejora = 100 * (f1_lem - f1_raw) / f1_raw
print(f"Mejora de F1-score: {f1_mejora:.2f}%")

# Análisis de negaciones comunes
negaciones = ['no', 'nunca', 'jamás', 'sin']
contador_neg = Counter()

for texto in df['lemas']:
    for palabra in texto.split():
        if palabra in negaciones:
            contador_neg[palabra] += 1

print("Palabras negativas más comunes:", contador_neg)


Reducción del vocabulario: 11.96%
Mejora de F1-score: -4.76%
Palabras negativas más comunes: Counter()


- Resultados finales

In [79]:
df.head(50)

,texto,idioma,sentimiento,lemas,sentimiento_binario
0,La interfaz intuitiva facilita la navegación i...,es,positivo,interfaz intuitivo facilitar navegación inclus...,1
1,El rendimiento gráfico dejó mucho que desear e...,es,negativo,rendimiento gráfico dejar desear juego exigente,0
2,Constantes fallos de conectividad WiFi tras la...,es,negativo,constante fallo conectividad wifi tras último ...,0
3,Sorprendido por la autonomía de la batería en ...,es,positivo,sorprendido autonomía batería modo uso intensivo,1
4,El sistema operativo se bloquea recurrentement...,es,negativo,sistema operativo bloquear recurrentemente eje...,0
5,Calidad de construcción premium que justifica ...,es,positivo,calidad construcción premium justificar precio...,1
6,El micrófono incorporado tiene una sensibilida...,es,negativo,micrófono incorporado tener sensibilidad defic...,0
7,Actualizaciones frecuentes que realmente mejor...,es,positivo,actualización frecuente realmente mejorar expe...,1
8,El sensor de huellas demora hasta 3 segundos e...,es,negativo,sensor huella demoro segundo reconocer dedo,0
9,Ergonomía excepcional para sesiones prolongada...,es,positivo,ergonomía excepcional sesión prolongado trabajo,1
